# Data
Input: Section 4 outputs (from 'match_gene_symbol_hugo.ipynb'): combined_recon3_ID_expression_W5D2T0_W5D2T120.txt, 
filtered_expression_data_W5D3T0.txt,
filtered_expression_data_W5D3T120.txt,
filtered_expression_data_W5D3T240.txt

Output: .mat files for each individual sample, organized into folders named based on their ID (W5D2T0, W5D2T120, W5D3T0, W5D3T120, W5D3T240). 
Each folder contains individual .mat files, representing context-specific models.

```matlab

%initialise the Cobra toolbox
initCobraToolbox(false); %don't update the toolbox
changeCobraSolver('gurobi', 'all');

%read in recon model
fileName='Recon3DModel.mat';
model=readCbModel(fileName);

% Set the objective function to ATP maintenance
ATP_M= find(~cellfun(@isempty, strfind(model.rxns, 'DM_atp_c_')));
model=changeObjective(model, model.rxns(ATP_M));

% Read in the combined gene expression data file
geneFile = textread('combined_recon3_ID_expression_W5D2T0_W5D2T120.txt');
geneNames = geneFile(:, 1);

% Process the gene expression data for W5D2T0, column 1 - 72 is T0, and column % 73 - 138 is T120
for i = 2:2:72 % Adjust indices for your data
    exprData.gene = cellstr(num2str(geneNames));
    exprData.value = geneFile(:, i);
    
    % Then it relates the gene names that you have to the gene in the model and
    % parses the data allowing you to link your gene expression data to the
    % gene-protein rules of the model
    [geneName, geneExp] = findUsedGenesLevels(model, exprData, 0);
    parsedGPR = model.rules;
    corrRxns = model.rxns;
    
    expressionCol = mapGeneToRxn(model, geneName, geneExp, parsedGPR, corrRxns);
    % Apply the GIMME algorithm, taking in the gene expression values, 
    % the threshold for gene expression level as a percentile of the whole 
    % .txt file (across all genes and samples)
    model_W5D2T0 = GIMME(model, expressionCol, prctile(geneFile(:, i), 75));
    
    % Write the generated model to a file
    writeCbModel(model_W5D2T0, 'fileName', ['W5D2T0_', num2str(i/2), '.mat']);
end


% Process the gene expression data for W5D2T120
 % Pull out the column of gene expression values for column i of the txt
    % file and make a exprData structure containing the gene names (extracted
    % above) and the gene expression values (column i)
for i = 73:73+66-1 % Adjust indices for your data
    exprData.gene = cellstr(num2str(geneNames));
    exprData.value = geneFile(:, i);

    % Then it relates the gene names that you have to the gene in the model and
    % parses the data allowing you to link your gene expression data to the
    % gene-protein rules of the model
    [geneName, geneExp] = findUsedGenesLevels(model, exprData, 0);
    parsedGPR = model.rules;
    corrRxns = model.rxns;
    
    expressionCol = mapGeneToRxn(model, geneName, geneExp, parsedGPR, corrRxns);
    
    % Apply the GIMME algorithm, taking in the gene expression values, 
    % the threshold for gene expression level as a percentile of the whole 
    % .txt file (across all genes and samples)
    model_W5D2T120 = GIMME(model, expressionCol, prctile(geneFile(:, i), 75));
    
    % Write the generated model to a file
    writeCbModel(model_W5D2T120, 'fileName', ['W5D2T120_', num2str(i-72+1), '.mat']);
end

% Optional: Sampling the solution space (commented out, use if needed)
% options.nStepsPerPoint = 1;
% options.nPointsReturned = 500;
% options.toRound = 1;
% [P_ind1, X1_ind1] = sampleCbModel(ind_1, [], [], options);

```